In [27]:
import eikon as ek
import pandas as pd
from sqlalchemy import create_engine
ek.set_app_key('150b92bce26746bbbee1258f4ac8453b6cb6a1ae')

In [93]:
df_tickers = pd.read_csv("CompaniesSample.csv")

In [48]:
tickers_lst = df_tickers['Ticker'].to_list()

fin_vars_lst = [
    'TR.TotalReturn.date',
    'TR.TtlCmnSharesOut', 
    'TR.TotalEquity', 
    'TR.TotalReturn',
    'TR.TotalDebtOutstanding',
    'TR.CapitalExpenditures',
    'TR.CurrentRatio'
    ]

In [69]:
df, err = ek.get_data(tickers_lst, fin_vars_lst, {'SDate': '2017-12-31', 'EDate': '2022-12-31', 'FRQ': 'FY', 'Curn': 'USD'})
df

,Instrument,Date,Total Common Shares Outstanding,Total Equity,Total Return,Total Debt,"Capital Expenditures, Cumulative",Current Ratio
0,CMPC.SN,2017-12-29T00:00:00Z,2500000000.0,8082116000.0,52.951184,4082462000.0,-405076000.0,2.32273
1,CMPC.SN,2018-12-28T00:00:00Z,2500000000.0,8241835000.0,8.073222,3808951000.0,-298923000.0,2.27579
2,CMPC.SN,2019-12-30T00:00:00Z,2500000000.0,8133310000.0,-13.143652,4065974000.0,-252058000.0,2.26496
3,CMPC.SN,2020-12-30T00:00:00Z,2500000000.0,7863289000.0,1.932665,4163993000.0,-167338000.0,2.99374
4,CMPC.SN,2021-12-30T00:00:00Z,2500000000.0,7546078000.0,-15.183907,4499243000.0,-224914000.0,2.06255
...,...,...,...,...,...,...,...,...
565,EPIRa.ST,2018-12-28T00:00:00Z,1213738700.0,2123378970.67462,-7.716015,653382133.658669,-117030409.832362,2.3406
566,EPIRa.ST,2019-12-30T00:00:00Z,1202952020.0,2431211279.6411,39.270775,899807733.390301,-109271523.178808,3.07425
567,EPIRa.ST,2020-12-30T00:00:00Z,1205924490.0,2884184642.35283,33.567351,1224132054.34095,-122340166.528704,3.64419
568,EPIRa.ST,2021-12-30T00:00:00Z,1206262770.0,2846380210.63811,57.39359,1016019116.73599,-102442694.043721,2.64528


In [70]:
df_fin = df_tickers[['Company Name', 'Ticker']].merge(df, right_on='Instrument', left_on='Ticker').drop(columns='Instrument')
df_fin['Date'] = df_fin['Date'].str[:-10] # Get rid of TZ info
df_fin = df_fin.loc[df_fin['Date'].isin(['2017-12-31','2018-12-31','2019-12-31','2020-12-31','2021-12-31'])].reset_index(drop=True)

In [84]:
engine = create_engine("sqlite:///GDS.db")
df_fin.to_sql('FinancialsTable', engine, index=False, if_exists="replace")

Calculations

Market Cap: Total shares * Value-Per-Share

In [ ]:
import yfinance as yf
import numpy as np

ticks = pd.read_csv("CompaniesYFinance.csv")
ticker = ticks.iloc[:, 1]
data = yf.download(ticker, start = , end = , interval = '1y')

closing_price = data['Close'].iloc[-1]

tot_shares = df_fin.iloc[:, 2]
vps = closing_price / tot_shares

mark_cap = np.multiply(tot_shares, vps)

ROE: (Total Return/Total Equity)*100

In [ ]:
tot_ret = df_fin.iloc[:, 4] 
tot_eq = df_fin.iloc[:, 3]

roe = np.dot(np.divide(tot_ret, tot_eq), 100)

Debt-to-Equity: Total Debt/Total Equity 

In [ ]:
tot_deb = df.fin.iloc[:, 5]

debt_eq = np.divide(tot_deb, tot_eq)